In [ ]:
import numpy as np
import pandas as pd

import skimage
import skimage.io
import skimage.filters
import skimage.morphology
from skimage.filters import threshold_otsu
from skimage import registration
from skimage.feature import ORB, match_descriptors
from skimage.transform import matrix_transform
from skimage import data
from skimage.feature import register_translation
from skimage.feature.register_translation import _upsampled_dft
from skimage.feature import peak_local_max
from skimage.measure import label
from skimage.morphology import closing, square
from skimage.measure import regionprops
from skimage.color import label2rgb

from scipy import ndimage as ndi
from scipy.ndimage import fourier_shift

from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.operation import decimate
from holoviews.plotting.util import process_cmap

import os
import glob

import bebi103

import colorcet

import bokeh
bokeh.io.output_notebook()

import holoviews as hv
hv.extension('bokeh')
bebi103.hv.set_defaults()

import panel as pn

from sklearn import linear_model

In [ ]:
# The directory containing the images
data_dir = '../data/200129_double_hit_spinowitz/mz13-1'

# glob string for images
im_glob = os.path.join(data_dir, '*')

# Get list of images
im_list = sorted(glob.glob(im_glob))

im_list

In [ ]:
dapi_raw = skimage.io.imread(im_list[0])
cfp_raw = skimage.io.imread(im_list[2])
chfp_raw = skimage.io.imread(im_list[4])

In [ ]:
def best_slice(dapi_raw, cfp_raw, chfp_raw):
    '''
    Calculates best slice of z-stack based on highest dapi+ area.
    Returns dapi, cfp, and chfp as the selected slice within the repsective channel.
    '''
    dapi_areas = []
    dapi_filt_gauss = skimage.filters.gaussian(dapi_raw, 1.5)
    threshold1 = threshold_otsu(dapi_filt_gauss)

    for i in range(len(dapi_raw)):
        dapi_filt_gauss_slice = dapi_filt_gauss[i]
        dapi_filt_gauss_bw = dapi_filt_gauss_slice > threshold1
        dapi_area = dapi_filt_gauss_bw.sum()
        dapi_areas.append(dapi_area)

    slices = [i for i in range(len(dapi_raw))]

    data = {'slice': slices, 'areas': dapi_areas}
    df_dapi = pd.DataFrame(data)

    hv.Points(data=df_dapi,
             kdims=['slice', 'areas'],
             )
    select_slice = dapi_areas.index(max(dapi_areas))
    print(select_slice)
    
    dapi = dapi_raw[select_slice]
    cfp = cfp_raw[select_slice]
    chfp = chfp_raw[select_slice]
    
    return dapi, cfp, chfp

I will now register the CFP and ChFP channels by finding the offset of ChFP, then translating/shifting the ChFP image accordingly.

In [ ]:
def register_images(cfp, chfp):
    '''
    Registers the two channels using register_translation from skimage.features.
    Prints detected pixel offset (y,x).
    Returns corrected moving array.
    '''
    # pixel precision first
    shift, error, diffphase = register_translation(cfp, chfp)
    print(f"Detected pixel offset (y, x): {shift}")
    chfp_shift = ndi.shift(chfp, shift)

    # corrected chfp_slice
    chfp = chfp_shift
    return chfp

I will define functions to display plots side by side.

In [ ]:
def show_two_ims(
    im_1,
    im_2,
    titles=[None, None],
    interpixel_distances=[0.13, 0.13],
    cmap=[None, None]
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range

    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

def show_three_ims(
    im_1,
    im_2,
    im_3,
    titles=[None, None, None],
    interpixel_distances=[0.13, 0.13, 0.13],
    cmap=[None, None, None],
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_3 = bebi103.image.imshow(
        im_3,
        frame_height=225,
        title=titles[2],
        cmap=cmap[2],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    p_3.x_range = p_1.x_range
    p_3.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2, p_3], ncols=3)

def show_four_ims(
    im_1,
    im_2,
    im_3,
    im_4,
    titles=[None, None, None, None],
    interpixel_distances=[0.13, 0.13, 0.13, 0.13],
    cmap=[None, None, None, None],
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_3 = bebi103.image.imshow(
        im_3,
        frame_height=225,
        title=titles[2],
        cmap=cmap[2],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_4 = bebi103.image.imshow(
        im_4,
        frame_height=225,
        title=titles[3],
        cmap=cmap[3],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    p_3.x_range = p_1.x_range
    p_3.y_range = p_1.y_range
    p_4.x_range = p_1.x_range
    p_4.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2, p_3, p_4], ncols=2)

def show_five_ims(
    im_1,
    im_2,
    im_3,
    im_4,
    im_5,
    titles=[None, None, None, None, None],
    interpixel_distances=[0.13, 0.13, 0.13, 0.13, 0.13],
    cmap=[None, None, None, None, None],
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_3 = bebi103.image.imshow(
        im_3,
        frame_height=225,
        title=titles[2],
        cmap=cmap[2],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_4 = bebi103.image.imshow(
        im_4,
        frame_height=225,
        title=titles[3],
        cmap=cmap[3],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_5 = bebi103.image.imshow(
        im_5,
        frame_height=225,
        title=titles[4],
        cmap=cmap[4],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    p_3.x_range = p_1.x_range
    p_3.y_range = p_1.y_range
    p_4.x_range = p_1.x_range
    p_4.y_range = p_1.y_range
    p_5.x_range = p_1.x_range
    p_5.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2, p_3, p_4, p_5], ncols=3)

def show_six_ims(
    im_1,
    im_2,
    im_3,
    im_4,
    im_5,
    im_6,
    titles=[None, None, None, None, None, None],
    interpixel_distances=[0.13, 0.13, 0.13, 0.13, 0.13, 0.13],
    cmap=[None, None, None, None, None, None],
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_3 = bebi103.image.imshow(
        im_3,
        frame_height=225,
        title=titles[2],
        cmap=cmap[2],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_4 = bebi103.image.imshow(
        im_4,
        frame_height=225,
        title=titles[3],
        cmap=cmap[3],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_5 = bebi103.image.imshow(
        im_5,
        frame_height=225,
        title=titles[4],
        cmap=cmap[4],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_6 = bebi103.image.imshow(
        im_6,
        frame_height=225,
        title=titles[5],
        cmap=cmap[5],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    p_3.x_range = p_1.x_range
    p_3.y_range = p_1.y_range
    p_4.x_range = p_1.x_range
    p_4.y_range = p_1.y_range
    p_5.x_range = p_1.x_range
    p_5.y_range = p_1.y_range
    p_6.x_range = p_1.x_range
    p_6.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2, p_3, p_4, p_5, p_6], ncols=3)

## Step 1. Filters

First, I will apply a gaussian filter.

In [ ]:
def filt_gauss(dapi, cfp, chfp, size):
    '''
    Applies a gaussian filter with supplied size using skimage.filters.gaussian.
    Returns filtered images.
    '''
    dapi_filt_gauss = skimage.filters.gaussian(dapi, 1.5)
    cfp_filt_gauss = skimage.filters.gaussian(cfp, 1.5)
    chfp_filt_gauss = skimage.filters.gaussian(chfp, 1.5)
    return dapi_filt_gauss, cfp_filt_gauss, chfp_filt_gauss

## Step 2. Thresholding

I will threshold using Otsu's method.

In [ ]:
def threshold(dapi_filt_gauss, cfp_filt_gauss, chfp_filt_gauss):
    '''
    Threshold given images based on Otsu's method.
    Returns thresholded images, as well as dapi_sum.
    '''
    threshold1 = threshold_otsu(dapi_filt_gauss)
    print(threshold1, 'is where the dapi cutoff point is.')
    dapi_filt_gauss_bw = dapi_filt_gauss > threshold1

    threshold2 = threshold_otsu(cfp_filt_gauss)
    print(threshold2, 'is where the cfp cutoff point is.')
    cfp_filt_gauss_bw = cfp_filt_gauss > threshold2

    threshold3 = threshold_otsu(chfp_filt_gauss)
    print(threshold3, 'is where the chfp cutoff point is.')
    chfp_filt_gauss_bw = chfp_filt_gauss > threshold3
    return dapi_filt_gauss_bw, cfp_filt_gauss_bw, chfp_filt_gauss_bw, threshold2, threshold3

In [ ]:
def dilate(dapi_filt_gauss_bw, cfp_filt_gauss_bw, chfp_filt_gauss_bw, size):
    '''
    Dilates given images with supplied size structuring element (disk shape)
    Returns dilated images.'''

    # Make the structuring element 1 pixel radius disk
    selem = skimage.morphology.disk(0)

    # Dilate image
    dapi_dil = skimage.morphology.dilation(dapi_filt_gauss_bw, selem)
    cfp_dil = skimage.morphology.dilation(cfp_filt_gauss_bw, selem)
    chfp_dil = skimage.morphology.dilation(chfp_filt_gauss_bw, selem)
    return dapi_dil, cfp_dil, chfp_dil

It seems like the dilation process has merged some nuclei. Let's try to use the watershed tool to separate the merged nuclei. I will do this by performing distance transformation, followed by identification of local maxima. I will then use these maxima as markers, for which I will perform the watershed. The watershed will be performed on the mask from the thresholded gaussian filter image.

At this point I will do watershed for dapi first, followed by cfp and chfp. I will use watershed method 1 (peak_local_max).

In [ ]:
def ws(chfp_dil, chfp):

    distance = ndi.distance_transform_edt(chfp_dil)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((25, 25)),
                                labels=chfp_dil)
    markers = ndi.label(local_maxi)[0]
    labels = skimage.morphology.watershed(-distance, markers, mask=chfp_dil, watershed_line=True)
    chfp_ws_mask = skimage.morphology.remove_small_objects(labels, min_size=50)
    label_image_chfp = skimage.measure.label(chfp_ws_mask)
    props_chfp = skimage.measure.regionprops_table(label_image_chfp, intensity_image=chfp, properties=('label',
                                                                                             'centroid',
                                                                                             'area',
                                                                                             'mean_intensity'))
    df_chfp = pd.DataFrame(props_chfp)
    chfp_area = df_chfp['area'].sum()
    return label_image_chfp, df_chfp, chfp_area

In [ ]:
def remove_large(label_image, df, intensity):
    empty_array = np.zeros_like(label_image)
    max_size = 1000

    for i, label in enumerate(df.loc[df.loc[:, 'area'] > 1000]['label']):
        x = label_image == label
        y = x * label
        empty_array += y
        
    large_sub = label_image - empty_array
    
    props_large = skimage.measure.regionprops_table(large_sub, intensity_image=intensity, properties=('label',
                                                                                         'centroid',
                                                                                         'area',
                                                                                         'mean_intensity'))
    df_large = pd.DataFrame(props_large)
    area_large = df_large['area'].sum()
    return large_sub, df_large

In [ ]:
def coloc(cfp_large_sub, chfp_large_sub, cfp, chfp):
    selem = skimage.morphology.disk(3)
    cfp_z4_e = skimage.morphology.erosion(cfp_large_sub, selem)
    chfp_z4_e = skimage.morphology.erosion(chfp_large_sub, selem)
    c_ch = np.logical_or(cfp_z4_e, chfp_z4_e)
    c_ch_small = skimage.morphology.remove_small_objects(c_ch, min_size=50)
    c_ch_label = skimage.measure.label(c_ch_small)

    props_cfp_co = skimage.measure.regionprops_table(c_ch_label, intensity_image=cfp, properties=('label',
                                                                                             'centroid',
                                                                                             'area',
                                                                                             'mean_intensity'))
    df_cfp_co = pd.DataFrame(props_cfp_co)

    props_chfp_co = skimage.measure.regionprops_table(c_ch_label, intensity_image=chfp, properties=('label',
                                                                                             'centroid',
                                                                                             'area',
                                                                                             'mean_intensity'))
    df_chfp_co = pd.DataFrame(props_chfp_co)

    cfp_vals = df_cfp_co['mean_intensity'].values
    chfp_vals = df_chfp_co['mean_intensity'].values
    data = {'CFP mean intensity': cfp_vals, 'ChFP mean intensity': chfp_vals}
    df_co = pd.DataFrame(data)
    return df_co

## Now that I have defined my functions, I will call all of them.

In [ ]:
dapi, cfp, chfp = best_slice(dapi_raw, cfp_raw, chfp_raw)

In [ ]:
chfp = register_images(cfp, chfp)

In [ ]:
dapi_filt_gauss, cfp_filt_gauss, chfp_filt_gauss = filt_gauss(dapi, cfp, chfp, 1.5)

In [ ]:
dapi_filt_gauss_bw, cfp_filt_gauss_bw, chfp_filt_gauss_bw, threshold2, threshold3 = threshold(dapi_filt_gauss, cfp_filt_gauss, chfp_filt_gauss)

In [ ]:
dapi_dil, cfp_dil, chfp_dil = dilate(dapi_filt_gauss_bw, cfp_filt_gauss_bw, chfp_filt_gauss_bw, 0)

In [ ]:
label_image_dapi, df_dapi, dapi_area = ws(dapi_dil, dapi)
label_image_cfp, df_cfp, cfp_area = ws(cfp_dil, cfp)
label_image_chfp, df_chfp, chfp_area = ws(chfp_dil, chfp)

In [ ]:
df_chfp, chfp_area

In [ ]:
cfp_large_sub, df_cfp_large = remove_large(label_image_cfp, df_cfp, cfp)
chfp_large_sub, df_chfp_large = remove_large(label_image_chfp, df_chfp, chfp)

In [ ]:
print('Percent CFP+ area:', cfp_area/dapi_area)
print('Percent ChFP+ area:', chfp_area/dapi_area)

In [ ]:
df_co = coloc(cfp_large_sub, chfp_large_sub, cfp, chfp)

In [ ]:
all_plot = hv.Points(
    data=df_co,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='All nuclei'
).opts()
all_plot

In [ ]:
x = df_co['CFP mean intensity'].values.reshape((-1, 1))
y = df_co['ChFP mean intensity'].values
model = linear_model.LinearRegression().fit(x, y)
r_sq = model.score(x, y)
print('R squared value is:', r_sq)

In [ ]:
cfp_thresh = threshold2 * 65535
chfp_thresh = threshold3 * 65535
print(cfp_thresh, chfp_thresh)

In [ ]:
df_cfp_chfp = df_co.loc[(df_co.loc[:, 'CFP mean intensity'] > cfp_thresh) &
                          (df_co.loc[:, 'ChFP mean intensity'] > chfp_thresh)]
df_cfp = df_co.loc[(df_co.loc[:, 'CFP mean intensity'] > cfp_thresh) &
                          (df_co.loc[:, 'ChFP mean intensity'] < chfp_thresh)]
df_chfp = df_co.loc[(df_co.loc[:, 'CFP mean intensity'] < cfp_thresh) &
                          (df_co.loc[:, 'ChFP mean intensity'] > chfp_thresh)]

In [ ]:
all_plot = hv.Points(
    data=df_co,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='All nuclei'
).opts(color='black', alpha=0.6)

cfp_plot = hv.Points(
    data=df_cfp,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='CFP+ only nuclei'
).opts(color=colorcet.glasbey_dark[3], alpha=0.6)

chfp_plot = hv.Points(
    data=df_chfp,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='mCh+ only Nuclei'
).opts(color=colorcet.glasbey_dark[13], alpha=0.6)

cfp_chfp_plot = hv.Points(
    data=df_cfp_chfp,
    kdims=['CFP mean intensity', 'ChFP mean intensity'],
    vdims=[],
    label='CFP+/mCh+ nuclei'
).opts(color=colorcet.glasbey_dark[4], alpha=0.8)

chfp_plot*cfp_plot*cfp_chfp_plot

In [ ]:
cfp_line = np.linspace(1000, 9000, 200)
chfp_line = np.linspace(500, 1400, 200)

cfp = (cfp_thresh, cfp_line)
chfp = (chfp_line, chfp_thresh)

c = hv.Path(cfp).opts(line_color='black', 
                          line_dash='dotted', 
                          line_alpha=0.5)
ch = hv.Path(chfp).opts(line_color='black', 
                          line_dash='dotted', 
                          line_alpha=0.5)

In [ ]:
chfp_plot*cfp_plot*cfp_chfp_plot*c*ch

In [ ]:
total_cells = len(df_co.index)
co_cells = len(df_cfp_chfp.index)
cfp_only_cells = len(df_cfp.index)
chfp_only_cells = len(df_chfp.index)

In [ ]:
print('Total nuclei:', total_cells)
print('Number of CFP+/mCh+ cells:', co_cells)
print('Number of CFP+ only cells:', cfp_only_cells)
print('Number of mCh+ only cells:', chfp_only_cells)

In [ ]:
print('% of CFP+/mCh+ cells:', co_cells / total_cells * 100)
print('% of CFP+ only cells:', cfp_only_cells / total_cells * 100)
print('% of mCh+ only cells:', chfp_only_cells / total_cells * 100)